In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from scipy.optimize import curve_fit
from lmfit import Model, minimize, Parameters, report_fit
from lmfit.models import GaussianModel, PolynomialModel
import astropy.units as u
import pandas as pd
import glob
import pandas as pd
import csv
import celerite

In [2]:
def Gauss(x, h1, c1, w1):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2))
    return G

def doubleGauss(x, h1, c1, w1, h2, c2, w2):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2))
    return G

def tripleGauss(x, h1, c1,w1, h2, c2, w2, h3, c3, w3):
    G = h1*np.exp(-(x-c1)**2/(2*w1**2)) + h2*np.exp(-(x-c2)**2/(2*w2**2)) + h3*np.exp(-(x-c3)**2/(2*w3**2))
    return G


In [3]:
def fit_coord(l, b, n_HI = 1, n_65 = 1, n_67 = 1, n_CO = 1, HI_p0 = [1, 0, 1], OH65_p0 = [0.03, 0, 4], OH67_p0 = [0.03, 0, 4], CO_p0 = [1, 0, 1], HI_bounds = (-np.inf, np.inf), OH65_bounds = (-np.inf, np.inf), OH67_bounds = (-np.inf, np.inf), CO_bounds = (-np.inf, np.inf)):
    
    # Setup a pandas dataframe to save all the variables we'd like
    
    '''Purpose:
    This function fits multiple gaussians to HI, CO and HI data and returns the parameters...
    
    Parameters:
    l: galactic latitude, int
    b: galactic lontitude, int
    HIdata: np.array of HI data in Tb, vel
    OHdata: np.array of OH data in Tb, vel
    ...

    Returns a pandas dataframe
    '''
    OH67 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_67basesub.txt')
    OH65 = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/OH' + str(l) + 'p' + str(b) + '_65basesub.txt')
    HI = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/HI' + str(l) + 'p' + str(b) + '.txt', data_start = 13, data_end = 167)
    CO = ascii.read('C:/Users/seven/OneDrive/Documents/DustSurvey/CO' + str(l) + 'p' + str(b) + '.txt')



    resultsTable = pd.DataFrame(np.array([[0, 
                                         0, 
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0,
                                         0]]),
                               columns=['l (deg)', 
                                        'b (deg)', 
                                        'line',
                                        'comp',
                                        'T_peak (K)',
                                        'T_error (K)', 
                                        'CenterVel (km/s)',
                                        'CenterVel_error (km/s)',
                                        'FWHM (km/s)',
                                        'FWHM error (km/s)'
                                       ])

    velHI = HI['col1']
    TaHI = HI['col2']
    vel67 = OH67['col1']
    Ta67 = OH67['col2']
    vel65 = OH65['col1']
    Ta65 = OH65['col2']
    velCO = CO['col1']
    TaCO = CO['col2']

    #fit HI data
    if n_HI == 1:

        parametersHI, covarianceHI = curve_fit(Gauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0]]
        fitHIc = [parametersHI[1]]
        fitHIw = [parametersHI[2]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0]]
        errorHIc = [perrHI[1]]
        errorHIw = [perrHI[2]]
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        ]

        
    if n_HI == 2:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        
        parametersHI, covarianceHI = curve_fit(doubleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)

        fitHIh = [parametersHI[0], parametersHI[3]]
        fitHIc = [parametersHI[1], parametersHI[4]]
        fitHIw = [parametersHI[2], parametersHI[5]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3]]
        errorHIc = [perrHI[1], perrHI[4]]
        errorHIw = [perrHI[2], perrHI[5]]
        
        
        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        ]
            
    if n_HI == 3:
        
        HI_p0 = np.tile(HI_p0, n_HI)
        HI_bounds = np.tile(HI_bounds, n_HI)
        parametersHI, covarianceHI = curve_fit(tripleGauss, velHI, TaHI, HI_p0, bounds = HI_bounds)
        
        fitHIh = [parametersHI[0], parametersHI[3], parametersHI[6]]
        fitHIc = [parametersHI[1], parametersHI[4], parametersHI[7]]
        fitHIw = [parametersHI[2], parametersHI[5], parametersHI[8]]
        perrHI = np.sqrt(np.diag(covarianceHI))
        errorHIh = [perrHI[0], perrHI[3], perrHI[6]]
        errorHIc = [perrHI[1], perrHI[4], perrHI[7]]
        errorHIw = [perrHI[2], perrHI[5], perrHI[8]]

        for i in range(n_HI):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'HI',
                                                        i+1,
                                                        fitHIh[i],
                                                        errorHIh[i],
                                                        fitHIc[i],
                                                        errorHIc[i],
                                                        fitHIw[i],
                                                        errorHIw[i],
                                                        ]

            
    #fit 65 data
    
    if n_65 == 1:
        parameters65, covariance65 = curve_fit(Gauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
        
        fit65h = [parameters65[0]]
        fit65c = [parameters65[1]]
        fit65w = [parameters65[2]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0]]
        error65c = [perr65[1]]
        error65w = [perr65[2]]

        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        ]
    
    if n_65 == 2:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        
        parameters65, covariance65 = curve_fit(doubleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)
    
        fit65h = [parameters65[0], parameters65[3]]
        fit65c = [parameters65[1], parameters65[4]]
        fit65w = [parameters65[2], parameters65[5]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3]]
        error65c = [perr65[1], perr65[4]]
        error65w = [perr65[2], perr65[5]]
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        ]

            
    if n_65 == 3:
        
        OH65_p0 = np.tile(OH65_p0, n_65)
        parameters65, covariance65 = curve_fit(tripleGauss, vel65, Ta65, OH65_p0, bounds = OH65_bounds)    
        
        fit65h = [parameters65[0], parameters65[3], parameters65[6]]
        fit65c = [parameters65[1], parameters65[4], parameters65[7]]
        fit65w = [parameters65[2], parameters65[5], parameters65[8]]
        perr65 = np.sqrt(np.diag(covariance65))
        error65h = [perr65[0], perr65[3], perr65[6]]
        error65c = [perr65[1], perr65[4], perr65[7]]
        error65w = [perr65[2], perr65[5], perr65[8]]
        
        
        for i in range(n_65):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '65',
                                                        i+1,
                                                        fit65h[i],
                                                        error65h[i],
                                                        fit65c[i],
                                                        error65c[i],
                                                        fit65w[i],
                                                        error65w[i],
                                                        ]


    #fit 67 data
    
    if n_67 == 1:
        parameters67, covariance67 = curve_fit(Gauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)
        
        fit67h = [parameters67[0]]
        fit67c = [parameters67[1]]
        fit67w = [parameters67[2]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0]]
        error67c = [perr67[1]]
        error67w = [perr67[2]]
        
        
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        ]

        
    if n_67 == 2:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        parameters67, covariance67 = curve_fit(doubleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds)

        fit67h = [parameters67[0], parameters67[3]]
        fit67c = [parameters67[1], parameters67[4]]
        fit67w = [parameters67[2], parameters67[5]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3]]
        error67c = [perr67[1], perr67[4]]
        error67w = [perr67[2], perr67[5]]

                
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        ]
            

            
    if n_67 == 3:
        
        OH67_p0 = np.tile(OH67_p0, n_67)
        parameters67, covariance67 = curve_fit(tripleGauss, vel67, Ta67, OH67_p0, bounds = OH67_bounds) 
    
        fit67h = [parameters67[0], parameters67[3], parameters67[6]]
        fit67c = [parameters67[1], parameters67[4], parameters67[7]]
        fit67w = [parameters67[2], parameters67[5], parameters67[8]]
        perr67 = np.sqrt(np.diag(covariance67))
        error67h = [perr67[0], perr67[3], perr67[6]]
        error67c = [perr67[1], perr67[4], perr67[7]]
        error67w = [perr67[2], perr67[5], perr67[8]]

                
        for i in range(n_67):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        '67',
                                                        i+1,
                                                        fit67h[i],
                                                        error67h[i],
                                                        fit67c[i],
                                                        error67c[i],
                                                        fit67w[i],
                                                        error67w[i],
                                                        ]

            
    #fit CO data
    
    if n_CO == 1:
        parametersCO, covarianceCO = curve_fit(Gauss, velCO, TaCO, CO_p0,bounds =  CO_bounds)
            
        fitCOh = [parametersCO[0]]
        fitCOc = [parametersCO[1]]
        fitCOw = [parametersCO[2]]
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0]]
        errorCOc = [perrCO[1]]
        errorCOw = [perrCO[2]]
               
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        ]

        
    if n_CO == 2:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        parametersCO, covarianceCO = curve_fit(doubleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds)
    
        fitCOh = [parametersCO[0], parametersCO[3]]
        fitCOc = [parametersCO[1], parametersCO[4]]
        fitCOw = [parametersCO[2], parametersCO[5]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3]]
        errorCOc = [perrCO[1], perrCO[4]]
        errorCOw = [perrCO[2], perrCO[5]]
    
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        ]

    if n_CO == 3:
        
        CO_p0 = np.tile(CO_p0, n_CO)
        parametersCO, covarianceCO = curve_fit(tripleGauss, velCO, TaCO, CO_p0, bounds = CO_bounds) 
            
        fitCOh = [parametersCO[0], parametersCO[3], parametersCO[6]]
        fitCOc = [parametersCO[1], parametersCO[4], parametersCO[7]]
        fitCOw = [parametersCO[2], parametersCO[5], parametersCO[8]] 
        perrCO = np.sqrt(np.diag(covarianceCO))
        errorCOh = [perrCO[0], perrCO[3], perrCO[6]]
        errorCOc = [perrCO[1], perrCO[4], perrCO[7]]
        errorCOw = [perrCO[2], perrCO[5], perrCO[8]]
        
        
        for i in range(n_CO):
            resultsTable.loc[len(resultsTable.index)] = [ l,
                                                        b,
                                                        'CO',
                                                        i+1,
                                                        fitCOh[i],
                                                        errorCOh[i],
                                                        fitCOc[i],
                                                        errorCOc[i],
                                                        fitCOw[i],
                                                        errorCOw[i],
                                                        ]


               
    resultsTable = resultsTable.drop(0)
    return resultsTable
    #df = fit_coord(...)
    #df.save_csv()

    #files = glob.glob(*.csv)
    #for file in files:
        #df.read_csv(...)
        #df_full.concatenate(df)
        #df_full.summary()
        
    
'''
SETTING UP THE BIG TABLE:

df = fit_coord(l, b, params)
df.to_csv("l+b_fitted.csv")
df

ELSEWHERE (another cell/notebook)

files = glob.glob(fits/*csv)

df_full = pd.df()

#concate each df:
for file in files:
    df.read_csv(file)
    df_full.concatenate(file)
'''

   
        
        
        
        
        
        
        

'\nSETTING UP THE BIG TABLE:\n\ndf = fit_coord(l, b, params)\ndf.to_csv("l+b_fitted.csv")\ndf\n\nELSEWHERE (another cell/notebook)\n\nfiles = glob.glob(fits/*csv)\n\ndf_full = pd.df()\n\n#concate each df:\nfor file in files:\n    df.read_csv(file)\n    df_full.concatenate(file)\n'

In [4]:
#fit_coord(119070, 2800, n_HI = 2)
df = fit_coord(119070, 2800, n_HI = 2)
df.to_csv("119070p2800_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,119070,2800,HI,1,6.805905,0.134199,-9.986013,0.218255,11.427910,0.200720
2,119070,2800,HI,2,25.108946,0.262239,-4.629767,0.021005,1.847756,0.024123
3,119070,2800,65,1,0.043520,0.002106,-4.251790,0.060399,1.081164,0.060425
4,119070,2800,67,1,0.086602,0.002094,-4.358241,0.030806,1.103239,0.030814
5,119070,2800,CO,1,1.613065,0.125558,-3.779779,0.077823,0.865799,0.077823


In [5]:
df = fit_coord(119210, 2000, n_HI = 3, n_67 = 2, OH67_p0 = (1, -1, 9), HI_p0 = (1, -8, 30),
          OH67_bounds = (-10, 10), 
          HI_bounds = ([0, -25, 3], [80, 25, 30]))
df.to_csv("119210p2000_fitted.csv")
df

#work on this fit a bit more

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,119210,2000,HI,1,25.231412,0.552844,-11.512670,0.112793,3.000000,0.102499
2,119210,2000,HI,2,21.267959,0.471899,-1.436886,0.157771,4.015097,0.164382
3,119210,2000,HI,3,4.384096,0.216440,-24.031410,1.531909,24.926262,1.177009
4,119210,2000,65,1,0.015542,0.002313,0.695469,0.242541,1.411649,0.242541
5,119210,2000,67,1,0.014841,0.001698,-2.937284,0.645761,2.897210,0.538266
6,119210,2000,67,2,0.038054,0.004263,0.625268,0.075415,0.747967,0.102232
7,119210,2000,CO,1,0.640718,0.144064,0.650001,0.190147,0.732573,0.190346


In [6]:
#fit_coord(120500, 1863, n_HI = 3)
df = fit_coord(120500, 1863, n_HI = 3, HI_p0 = (1, 1, 1), 
          HI_bounds = ([0, -12, 1], [30, 5, 50]))
df.to_csv("120500p1863_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,120500,1863,HI,1,16.151980,1.153345,-9.055389,0.377908,9.278406,0.337238
2,120500,1863,HI,2,29.351272,1.058692,-1.140389,0.061317,1.947711,0.081703
3,120500,1863,HI,3,7.999107,1.201906,-9.832292,0.209921,1.888819,0.323243
4,120500,1863,65,1,0.056665,0.002598,-0.806004,0.046752,0.882485,0.046676
5,120500,1863,67,1,0.098978,0.002308,-0.888053,0.026717,0.991228,0.026658
6,120500,1863,CO,1,2.971228,0.136213,-0.813725,0.036567,0.690755,0.036565


In [9]:
df = fit_coord(120500, 2963, n_67 = 2, OH67_p0 = [1, 0, 2], CO_p0 = [1, -5, 0.5], 
           OH65_p0 = [-1, -8, 1], OH67_bounds = (-12,5), OH65_bounds = ([-1, -25, 0], [0, 0, 5]))
df.to_csv("120500p2963_fitted.csv")
df

#look into 65 fit

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,120500,2963,HI,1,22.742624,0.609856,-4.095128,0.146664,4.735093,0.146632
2,120500,2963,65,1,-0.083266,0.005933,-8.052769,0.057591,0.176147,0.020393
3,120500,2963,67,1,0.014573,0.001355,-3.185613,0.462390,3.041731,0.372444
4,120500,2963,67,2,0.030205,0.003265,-6.391264,0.075614,0.627062,0.089212
5,120500,2963,CO,1,1.475331,0.128105,-6.307529,0.057041,0.563438,0.056081


In [10]:
#fit_coord(121070, 2175, n_65 = 3, n_67 = 2, n_CO = 2, n_HI = 2, 
          #CO_bounds = ([0, -10, 0, 0, -10, 0],[5, 1, 5, 5, 1, 5]))
df = fit_coord(121070, 2175, n_65 = 3, n_67 = 2, n_CO = 2, n_HI = 2, 
          CO_bounds = ([0, -10, 0, 0, -10, 0],[5, 1, 5, 5, 1, 5]))
df.to_csv("121070p2175_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,121070,2175,HI,1,9.894511,0.414991,-6.992067,0.301417,9.146875,0.253443
2,121070,2175,HI,2,9.199639,0.514995,-2.795857,0.134952,2.573737,0.172624
3,121070,2175,65,1,0.082024,0.003625,-5.878189,0.035285,0.657108,0.038387
4,121070,2175,65,2,0.022957,0.002715,-2.661048,0.261290,1.791566,0.325637
5,121070,2175,65,3,-0.007852,0.002341,-5.415900,1.235540,5.771678,1.032767
6,121070,2175,67,1,0.122155,0.004858,-6.000259,0.022379,0.666106,0.044353
7,121070,2175,67,2,0.036667,0.001859,-3.012325,0.276424,2.051407,0.236955
8,121070,2175,CO,1,3.861257,0.274409,-6.122722,0.043057,0.780493,0.048677
9,121070,2175,CO,2,1.935918,0.091977,-3.416228,0.234138,1.650391,0.196384


In [21]:
#fit_coord(121930, 1975, n_HI = 2, n_67 = 2, OH67_p0 = (1, 0, 8), HI_p0 = (1.3, -10, 4),
           #OH65_p0 = (1, 0, 4), 
           #OH65_bounds = ([0, -2, .1], [1, 2, 4]), 
           #OH67_bounds = ([0, -10, .1, 0, -10, .1], [20, 5, 20, 20, 5, 20]), 
           #HI_bounds = ([0, -20, 1, 0, -10, 1], [40, 0, 50, 40, 0, 50])) 
#check later
#bounds issue

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,121930,1975,HI,1,18.323960,0.657324,-12.521184,0.460614,5.477664,0.425020
2,121930,1975,HI,2,19.510660,1.101173,-0.612585,0.303870,3.564324,0.254745
3,121930,1975,65,1,0.060048,988.940778,1.298663,82.403629,0.252081,1228.486705
4,121930,1975,67,1,0.015215,0.002820,-2.997574,0.160875,0.717521,0.149671
5,121930,1975,67,2,0.118112,1300.779045,1.323984,10.731620,0.249545,798.576881
6,121930,1975,CO,1,0.859033,0.160622,1.332860,0.119604,0.569361,0.125836


In [9]:
df = fit_coord(122220, 1875, n_HI = 3, HI_p0 = (1, -10, 25), CO_p0 = (1, 2, 0.5),
          CO_bounds = ([0, 0, 0.1], [3, 3, 3]), 
          HI_bounds = ([0, -20, 1], [30, 20, 50]))
df.to_csv("122220p1875_fitted.csv")
df
#CHECK LATER ON CO ERROR (basically just noise as seen in plot)

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,122220,1875,HI,1,10.443772,1.629264,-12.628355,0.225800,1.316190,0.258288
2,122220,1875,HI,2,19.828186,1.001218,-8.541770,0.541676,9.482401,0.378219
3,122220,1875,HI,3,5.199265,1.425286,0.066284,0.683497,2.496087,0.858417
4,122220,1875,65,1,0.013580,0.001910,1.981341,0.187442,1.154046,0.187441
5,122220,1875,67,1,0.027175,0.002403,1.999606,0.082510,0.803392,0.081722
6,122220,1875,CO,1,2.180064,4531.473308,1.964676,19.727884,0.192040,139.530787


In [17]:
fit_coord(122500, 2900, n_HI = 2, n_67 = 3, HI_p0 = (1, -10, 25), OH67_p0 = (1, -4, 4),
         OH67_bounds = (-15, 4), CO_bounds = (-12, 1))
#issue with OH bounds and CO 

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,122500,2900,HI,1,17.644321,0.443367,-5.238368,0.119842,3.936416,0.125039
2,122500,2900,HI,2,9.587270,0.401918,-22.515761,0.243541,4.879170,0.259820
3,122500,2900,65,1,0.012392,0.001657,-6.669039,0.205526,1.331156,0.205526
4,122500,2900,67,1,0.446464,1737.664811,-7.757856,81.430769,3.428040,58.060027
5,122500,2900,67,2,-0.441831,1737.664968,-7.799206,80.372086,3.398016,59.421415
6,122500,2900,67,3,0.012986,0.003640,-7.225295,0.186731,0.537768,0.261347
7,122500,2900,CO,1,-0.582909,42620.352991,-1.258068,1747.729971,0.172326,3599.875856


In [22]:
fit_coord(123360, 2075, n_HI = 3, HI_p0 = (2, -3, 1), n_65 = 2, OH65_p0 = (1, 0, 6), 
          n_67 = 3, OH67_p0 = (1, 0, 4), n_CO = 2, CO_p0 = (1, 0, 3)
         , OH67_bounds = (-5, 12))
#67 temperature peak questionable

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,123360,2075,HI,1,18.089079,0.598007,-12.401703,0.341561,5.083411,0.340006
2,123360,2075,HI,2,19.038641,0.766523,4.564560,0.175267,3.071973,0.192282
3,123360,2075,HI,3,25.697196,1.364159,-3.592400,0.114687,1.981034,0.135203
4,123360,2075,65,1,0.013952,0.001487,-0.045980,0.472615,3.121495,0.357244
5,123360,2075,65,2,0.100461,376.192674,3.043848,78.569559,0.258018,307.651493
6,123360,2075,67,1,-1.794111,2007.650278,2.700439,4.631984,0.984828,8.737440
7,123360,2075,67,2,0.034925,0.008170,0.959767,0.403031,2.540167,0.175548
8,123360,2075,67,3,1.834668,2007.656173,2.708605,4.410905,0.969385,8.360303
9,123360,2075,CO,1,1.623679,0.091749,-0.481574,0.086187,1.122122,0.094519
10,123360,2075,CO,2,1.423822,0.106477,2.725460,0.083832,0.792729,0.084513


In [4]:
df = fit_coord(123500, 1963, n_67 = 2, n_HI = 3, n_CO = 2, HI_p0 = (15, 0, 5),
           CO_p0 = (1, -4, 8), OH65_bounds = (-12, 20), CO_bounds = (-10, 10), 
           HI_bounds= ([10, -30, 1], [80, 25, 15]))
df.to_csv("123500p1963_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,123500,1963,HI,1,18.089110,0.598024,-12.401713,0.341525,5.083383,0.339978
2,123500,1963,HI,2,25.697267,1.364069,-3.592402,0.114687,1.981038,0.135201
3,123500,1963,HI,3,19.038646,0.766519,4.564559,0.175269,3.071973,0.192284
4,123500,1963,65,1,0.021746,0.001509,-3.947268,0.135462,1.690208,0.135462
5,123500,1963,67,1,0.034361,0.001515,-3.964760,0.075600,1.484736,0.075616
6,123500,1963,67,2,0.008113,0.001459,5.086174,0.332476,1.600282,0.332550
7,123500,1963,CO,1,0.527750,0.186116,-4.651612,0.416159,1.433614,0.298316
8,123500,1963,CO,2,0.405407,0.219061,-5.356205,0.223275,0.466556,0.298616


In [5]:
df = fit_coord(125070, 1950, n_HI = 2)
df.to_csv("125070p1950_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,125070,1950,HI,1,16.459060,0.269667,-8.097327,0.399493,10.081815,0.284662
2,125070,1950,HI,2,8.403681,0.719002,4.079510,0.195456,3.464993,0.308161
3,125070,1950,65,1,0.059559,0.002486,0.686141,0.038342,0.555600,0.025695
4,125070,1950,67,1,0.106404,0.002872,0.660790,0.022660,0.620396,0.018588
5,125070,1950,CO,1,1.962675,0.147545,0.680557,0.051534,0.601743,0.052906


In [7]:
df = fit_coord(125220, 3250, OH67_bounds = (-10, 5))
df.to_csv("125220p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,125220,3250,HI,1,36.662619,0.776355,1.152745,0.067798,2.775750,0.067794
2,125220,3250,65,1,0.068332,0.002120,0.234129,0.034403,0.961648,0.034518
3,125220,3250,67,1,0.120392,0.002196,0.069041,0.022092,1.048731,0.022080
4,125220,3250,CO,1,0.863866,0.103696,0.389683,0.137698,0.993339,0.137698


In [8]:
df = fit_coord(125500, 2200, n_HI = 2, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 2)
         , OH67_bounds = (-10, 10)) 
df.to_csv("125500p2200_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,125500,2200,HI,1,14.435092,0.377689,-2.672260,0.092367,3.392171,0.111447
2,125500,2200,HI,2,6.078892,0.242599,-9.139887,0.554405,16.841811,0.536649
3,125500,2200,65,1,0.029502,0.003612,-4.582641,0.082615,0.594410,0.099513
4,125500,2200,65,2,0.009467,0.002535,-3.344009,0.598634,2.205889,0.426794
5,125500,2200,67,1,0.016240,0.002329,-3.185738,0.356943,2.474474,0.250410
6,125500,2200,67,2,0.056399,0.003244,-4.709780,0.041191,0.739604,0.044936
7,125500,2200,CO,1,2.437001,0.095121,-4.804518,0.038853,0.911051,0.043082


In [10]:
#df = 
fit_coord(127500, 2088, n_67 = 2, n_65 = 2, n_HI = 3, OH67_p0 = (1, 0, 6), OH65_p0 = (1, 0, 6), 
          HI_p0 = (1, 0, 10), OH65_bounds = (-10, 25), OH67_bounds = (-15, 15))
#df.to_csv("127500p2088_fitted.csv")
#df

ValueError: too many values to unpack (expected 2)

In [10]:
df = fit_coord(137160, 4300, OH67_p0 = (1, 10, 1), n_HI = 2,
          HI_bounds = ([0, 0, 1, 0, 0, 1], [20 ,10, 10, 20 ,12, 10]))
df.to_csv("137160p4300_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,137160,4300,HI,1,2.703331e+00,1.647863e-01,4.816471e-02,7.787936e-01,10.000000,6.141708e-01
2,137160,4300,HI,2,1.123248e+01,3.641153e-01,7.866307e+00,6.185188e-02,1.753286,7.117435e-02
3,137160,4300,65,1,3.101224e-03,1.551295e-03,-1.008849e+00,9.413538e-01,1.629755,9.413538e-01
4,137160,4300,67,1,1.004865e-02,2.068852e-03,8.702887e+00,1.959348e-01,0.823589,1.956942e-01
5,137160,4300,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10


In [15]:
df = fit_coord(142210, 2325, n_HI = 3, HI_p0 = (1, 0, 8), 
           HI_bounds = ([0, -30, 0], [20, 5, 50]))
df.to_csv("142210p2325_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,142210,2325,HI,1,0.818782,0.095787,-28.850766,5.007798,36.470092,3.877945
2,142210,2325,HI,2,14.290681,0.283359,-3.126089,0.110026,8.025420,0.147917
3,142210,2325,HI,3,19.395988,0.384563,-1.330761,0.033308,1.744758,0.042756
4,142210,2325,65,1,0.043649,0.002481,-1.955868,0.053688,0.789529,0.050298
5,142210,2325,67,1,0.084863,0.002742,-2.042813,0.031121,0.820334,0.030163
6,142210,2325,CO,1,2.504879,0.131268,-2.169310,0.043053,0.711052,0.043011


In [12]:
df = fit_coord(143000, 3850, n_HI = 2)
df.to_csv("143000p3850_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,143000,3850,HI,1,17.597814,1.611684,4.112784,0.276457,3.666823,0.175870
2,143000,3850,HI,2,12.805675,1.821219,2.037581,0.150877,1.359348,0.202028
3,143000,3850,65,1,0.053961,0.002037,2.292664,0.039967,0.921137,0.040368
4,143000,3850,67,1,0.097741,0.002296,2.225671,0.024816,0.918526,0.025030
5,143000,3850,CO,1,2.491926,0.101556,2.302071,0.037419,0.795076,0.037414


In [13]:
df = fit_coord(146070, 1775, n_HI = 2
               , OH67_bounds = ([0, -5, 0], [20, 5, 5]),
               OH65_bounds = ([0, -5, 0], [20, 5, 5]))
df.to_csv("146070p1775_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146070,1775,HI,1,22.857251,0.905702,0.666614,0.069938,2.126429,0.092529
2,146070,1775,HI,2,15.026091,0.804735,-2.406825,0.253294,6.331613,0.192466
3,146070,1775,65,1,0.169170,0.002339,0.591286,0.012319,0.687076,0.010227
4,146070,1775,67,1,0.230267,0.002680,0.487612,0.010358,0.723211,0.009325
5,146070,1775,CO,1,4.350812,0.114572,0.258126,0.027344,0.899220,0.027344


In [14]:
df = fit_coord(146200, 3963, n_HI = 2, HI_p0 = (1, 0, 5), HI_bounds = (-15, 15))
df.to_csv("146200p3963_fitted.csv")
df

#weird HI bounds behavior when varying upper bounds

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146200,3963,HI,1,15.000000,1.058495,5.287060,0.191374,1.869386,0.184717
2,146200,3963,HI,2,9.474446,0.798416,-2.466710,0.405901,3.180859,0.439862
3,146200,3963,65,1,0.017295,0.003077,0.399105,0.124299,0.695627,0.154214
4,146200,3963,67,1,0.022717,0.001933,-0.098259,0.122012,1.241909,0.122010
5,146200,3963,CO,1,1.779890,0.092886,0.001515,0.049196,0.816434,0.049200


In [15]:
df = fit_coord(146930, 2075)
df.to_csv("146930p2075_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,146930,2075,HI,1,37.110928,0.772866,-1.210916,0.072475,3.014247,0.072489
2,146930,2075,65,1,0.033501,0.009247,0.136408,0.152707,0.642267,0.228113
3,146930,2075,67,1,0.057882,0.005701,-0.134523,0.096376,0.841744,0.095200
4,146930,2075,CO,1,0.697665,0.092378,0.055885,0.106260,0.695648,0.106522


In [16]:
df = fit_coord(147000, 4038, n_CO = 2, n_67 = 2, n_65 = 2, OH67_bounds = (-8, 5),
               OH65_bounds = (-12, 5))
df.to_csv("147000p4038_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,147000,4038,HI,1,11.355956,0.258733,0.643146,0.131294,4.992655,0.131359
2,147000,4038,65,1,0.008834,0.002399,-0.918983,0.567634,3.257406,0.706341
3,147000,4038,65,2,0.016684,0.003595,-1.267393,0.147423,0.593570,0.184703
4,147000,4038,67,1,0.041632,0.002633,-1.883540,0.100931,0.942039,0.100082
5,147000,4038,67,2,0.007669,0.002651,1.023237,0.545528,0.931538,0.541062
6,147000,4038,CO,1,1.542826,0.100168,-1.863680,0.063116,0.841038,0.064972
7,147000,4038,CO,2,0.607143,0.128374,1.116350,0.126598,0.495887,0.123496


In [17]:
#df = fit_coord(147200, 4075, n_67 = 2, n_65 = 2, n_CO = 2, CO_bounds = (-10, 10),
               OH65_bounds = (-25,10))
#df.to_csv("147200p4075_fitted.csv")
#df

#issue when I apply bounds to 65

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,147200,4075,HI,1,19.663554,0.344352,1.742909,0.076426,3.782216,0.076468
2,147200,4075,65,1,0.119733,437.017328,4.040055,74.128994,0.290254,417.503106
3,147200,4075,65,2,0.016174,0.161087,3.114782,11.145493,0.819558,3.719517
4,147200,4075,67,1,0.025083,0.001708,-1.748461,0.088048,1.119444,0.088063
5,147200,4075,67,2,0.076728,0.002021,3.683673,0.023821,0.731117,0.021176
6,147200,4075,CO,1,3.999901,0.165460,3.718547,0.025175,0.634194,0.030279
7,147200,4075,CO,2,0.131452,0.074998,6.784164,1.907052,1.791054,1.951131


In [18]:
df = fit_coord(148200, 3863, n_HI = 2, HI_p0 = (1, 0, 5), 
               HI_bounds = ([0, -10, 1, 0, -10, 1], [20, 10, 20, 20, 10, 20]))
df.to_csv("148200p3863_fitted.csv")
df


,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,148200,3863,HI,1,8.934638,1.033549,1.100517,0.460988,5.912545,0.459897
2,148200,3863,HI,2,7.812903,1.254322,-0.933010,0.231440,1.572206,0.301958
3,148200,3863,65,1,0.041634,0.002530,-2.200168,0.060638,0.860289,0.060124
4,148200,3863,67,1,0.047720,0.002155,-1.827805,0.062944,1.207130,0.062943
5,148200,3863,CO,1,1.576454,0.102998,-2.316782,0.052379,0.693315,0.052232


In [19]:
df = fit_coord(148210, 2350, n_HI = 2, HI_p0 = (1, 0, 5), OH67_bounds = (-5, 5))
df.to_csv("148210p2350_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,148210,2350,HI,1,11.950790,0.233022,-1.813370,0.157707,8.475072,0.148829
2,148210,2350,HI,2,10.816938,0.469053,1.905386,0.061185,1.293280,0.070373
3,148210,2350,65,1,0.044365,0.003695,1.283639,0.064017,0.609142,0.059267
4,148210,2350,67,1,0.066329,0.003010,1.370475,0.038548,0.671339,0.033823
5,148210,2350,CO,1,3.165647,0.141303,1.304324,0.026811,0.545670,0.029165


In [20]:
df = fit_coord(150120, 4067)
df.to_csv("150120p4067_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,150120,4067,HI,1,1.060897e+01,2.619626e-01,1.221581e+00,1.480027e-01,5.192746,1.480601e-01
2,150120,4067,65,1,2.465818e-03,9.400724e-04,1.338215e+00,2.449608e+00,5.567353,2.449608e+00
3,150120,4067,67,1,3.396686e-03,1.080381e-03,-3.658921e+00,1.169826e+00,3.183563,1.169826e+00
4,150120,4067,CO,1,-1.617513e-19,2.212943e-20,1.130883e-09,1.319636e-10,1.000000,8.437336e-10


In [21]:
df = fit_coord(151500, 1925, n_HI = 2, HI_bounds = (-25, 20))
df.to_csv("151500p1925_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,151500,1925,HI,1,13.944617,0.757852,-1.122368,0.279192,8.748708,0.360988
2,151500,1925,HI,2,20.000000,1.010960,-0.746887,0.089165,1.846457,0.115769
3,151500,1925,65,1,0.011849,0.002383,-1.449924,0.192803,0.812572,0.185016
4,151500,1925,67,1,0.022660,0.001887,-1.235308,0.124834,1.298628,0.124834
5,151500,1925,CO,1,1.513998,0.124205,-1.399332,0.066468,0.702123,0.066562


In [22]:
df = fit_coord(156540, 3513, n_67 = 2, n_65 = 2, OH67_p0 = (1, 0, 3), OH65_p0 = (1, 0, 1)
               ,CO_bounds = (-10, 1), OH67_bounds = (-5, 8), OH65_bounds = (-5, 8))
df.to_csv("156540p3513_fitted.csv")
df
#weird CO behavior when bounds applied

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,156540,3513,HI,1,15.205414,0.417686,-0.789128,0.116018,3.658962,0.116042
2,156540,3513,65,1,0.005952,0.001734,-1.547909,0.818511,3.008808,0.711800
3,156540,3513,65,2,0.017492,0.002736,-3.061133,0.117392,0.651924,0.107105
4,156540,3513,67,1,0.017449,0.002465,0.426709,0.103878,0.666727,0.115559
5,156540,3513,67,2,0.032019,0.002018,-3.067125,0.064991,0.883920,0.065264
6,156540,3513,CO,1,1.000000,0.102797,-3.396298,0.118700,1.000000,0.118700


In [23]:
df = fit_coord(156640, 3250)
df.to_csv("156640p3250_fitted.csv")
df

,l (deg),b (deg),line,comp,T_peak (K),T_error (K),CenterVel (km/s),CenterVel_error (km/s),FWHM (km/s),FWHM error (km/s)
1,156640,3250,HI,1,23.456820,0.679447,-0.742946,0.095651,2.861469,0.095685
2,156640,3250,65,1,0.002425,0.000994,0.949288,1.781178,3.763955,1.781178
3,156640,3250,67,1,0.001190,0.001536,-0.204544,3.025401,-2.028822,3.025401
4,156640,3250,CO,1,1.742431,0.101835,-0.815809,0.068763,1.018795,0.068763


In [20]:
#fit_coord(157350, 2200, n_HI = 3, HI_p0 = (1, 0, 20)) null

In [29]:
files = glob.glob("C:/Users/seven/OneDrive/Documents/CO Dark Final csv/*csv*")

In [30]:
for file in files:
    df.read_csv(file)
    df_full.concatenate(file)
    df_full.summary
